In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2
%matplotlib inline
from __future__ import annotations
import sys

sys.path.append("../")

In [2]:
from mesoscaler._metadata import get_metadata

get_metadata()

[]

In [3]:
import mesoscaler.enums

get_metadata()  # metadata is lazy loaded via enum descriptors

[{'hash': 5884261301866,
  '__mesometa_cls_data__': {},
  '__mesometa_series__': {'T': T, 'Z': Z, 'Y': Y, 'X': X},
  '__mesometa_member_data__': {'T': {}, 'Z': {}, 'Y': {}, 'X': {}},
  '__mesometa_member_aliases__': [{'T': 't',
    'Z': 'height',
    'Y': 'y',
    'X': 'longitude'},
   {'T': 'time', 'Z': 'z', 'Y': 'grid_latitude', 'X': 'x'},
   {'T': None, 'Z': 'altitude', 'Y': 'latitude', 'X': 'grid_longitude'},
   {'T': None, 'Z': 'level', 'Y': None, 'X': None}]},
 {'hash': 5884261302131,
  '__mesometa_cls_data__': {},
  '__mesometa_series__': {'time': time,
   'vertical': vertical,
   'latitude': latitude,
   'longitude': longitude},
  '__mesometa_member_data__': {'time': {'axis': (T,)},
   'vertical': {'axis': (Z,)},
   'latitude': {'axis': (Y, X)},
   'longitude': {'axis': (Y, X)},
   'axis': {}},
  '__mesometa_member_aliases__': [{'time': None,
    'vertical': 'height',
    'latitude': 'grid_latitude',
    'longitude': 'grid_longitude'},
   {'time': None, 'vertical': 'latitude', 

In [4]:
import os
import types

import pyproj
import xarray as xr
import pyresample.kd_tree

from mesoscaler.core import make_independent, DERIVED_SFC_COORDINATE, GriddedDataset
from mesoscaler.typing import Any
from mesoscaler.enums import (
    URMA,
    ERA5,
    Dimensions,
    Coordinates,
    X,
    Y,
    Z,
    T,
    LAT,
    LON,
    LVL,
    TIME,
    COORDINATES,
    DIMENSIONS,
)

_test_data = "../tests/data"

urma_store = os.path.join(_test_data, "urma.zarr")
urma_dvars = list(URMA)

era5_store = os.path.join(_test_data, "era5.zarr")
era5_dvars = list(ERA5)

print(era5_dvars, urma_dvars, sep="\n")

[geopotential, specific_humidity, temperature, u_component_of_wind, v_component_of_wind, vertical_velocity]
[total_cloud_cover, ceiling, u_wind_component_10m, v_wind_component_10m, wind_speed_10m, wind_speed_gust, wind_direction_10m, temperature_2m, dewpoint_temperature_2m, specific_humidity_2m, surface_pressure, visibility, orography]


In [5]:
import pandas as pd

In [6]:
_URMA_DATASET = xr.open_zarr(urma_store)
_URMA_DATASET

<xarray.Dataset>
Dimensions:                  (time: 2, y: 1597, x: 2345)
Coordinates:
  * time                     (time) datetime64[ns] 2019-01-02 2019-01-02T01:0...
Dimensions without coordinates: y, x
Data variables: (12/14)
    ceiling                  (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    dewpoint_temperature_2m  (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    latitude                 (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
    longitude                (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
    specific_humidity_2m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    surface_pressure         (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    ...                       ...
    u_wind_component_10m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    visibility               (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_direction_10m       (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_speed_10m           (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_speed_gust          (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [7]:
if not os.path.exists(era5_store):
    _google_store = "gs://weatherbench2/datasets/era5/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2"
    xr.open_zarr(_google_store)[era5_dvars].sel(time=_URMA_DATASET.time).to_zarr(era5_store, mode="w")

_ERA5_DATASET = xr.open_zarr(era5_store)
_ERA5_DATASET

<xarray.Dataset>
Dimensions:              (time: 2, level: 37, latitude: 721, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 1 2 3 5 7 10 ... 875 900 925 950 975 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * time                 (time) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Data variables:
    geopotential         (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    specific_humidity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    u_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [8]:
ds = _URMA_DATASET.copy()

ds = ds.rename_dims(Dimensions.remap(ds.dims)).rename_vars(Coordinates.remap(ds.coords))
ds = ds.set_coords(Coordinates.intersection(ds.variables))
ds = ds.rename_vars(Coordinates.remap(ds.coords))
ds[LON], ds[LAT] = (ds[coord].compute() for coord in (LON, LAT))
# ds = ds.rename_vars(Coordinates.remap(ds.coords))
if missing_dims := Dimensions.difference(ds.dims):
    for dim in missing_dims:
        ds = ds.expand_dims(dim, axis=[DIMENSIONS.index(dim)])

if missing_coords := Coordinates.difference(ds.coords):
    assert missing_coords == {LVL}, missing_coords
    ds = ds.assign_coords(DERIVED_SFC_COORDINATE)

ds

<xarray.Dataset>
Dimensions:                  (T: 2, Z: 1, Y: 1597, X: 2345)
Coordinates:
    latitude                 (Y, X) float64 19.23 19.23 19.24 ... 54.38 54.37
    longitude                (Y, X) float64 233.7 233.7 233.8 ... 300.9 301.0
  * time                     (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
    vertical                 (Z) float64 1.013e+03
Dimensions without coordinates: T, Z, Y, X
Data variables:
    ceiling                  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    dewpoint_temperature_2m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    specific_humidity_2m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    surface_pressure         (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    temperature_2m           (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    total_cloud_cover        (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    u_wind_component_10m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    visibility               (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_direction_10m       (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_speed_10m           (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_speed_gust          (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [9]:
def get_urma() -> xr.Dataset:
    return make_independent(_URMA_DATASET.copy())


def get_era5() -> xr.Dataset:
    return make_independent(_ERA5_DATASET.copy())


urma = get_urma()
urma

<xarray.Dataset>
Dimensions:                  (T: 2, Z: 1, Y: 1597, X: 2345)
Coordinates:
    latitude                 (Y, X) float64 19.23 19.23 19.24 ... 54.38 54.37
    longitude                (Y, X) float64 233.7 233.7 233.8 ... 300.9 301.0
  * time                     (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
  * vertical                 (Z) float64 1.013e+03
Dimensions without coordinates: T, Z, Y, X
Data variables:
    ceiling                  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    dewpoint_temperature_2m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    specific_humidity_2m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    surface_pressure         (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    temperature_2m           (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    total_cloud_cover        (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    u_wind_component_10m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    visibility               (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_direction_10m       (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_speed_10m           (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_speed_gust          (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [10]:
era5 = get_era5()
era5

<xarray.Dataset>
Dimensions:              (T: 2, Z: 37, Y: 721, X: 1440)
Coordinates:
    latitude             (Y, X) float32 90.0 90.0 90.0 ... -90.0 -90.0 -90.0
  * vertical             (Z) int64 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
    longitude            (Y, X) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time                 (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Dimensions without coordinates: T, Z, Y, X
Data variables:
    geopotential         (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    specific_humidity    (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature          (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    u_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity    (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [11]:
ds = GriddedDataset.from_zarr(urma_store, [URMA.CEIL, URMA.VIS])
print(ds.crs.is_geocentric, ds.crs.is_geographic, ds.crs.is_projected)
ds = GriddedDataset.from_zarr(era5_store, [ERA5.T])
print(ds.crs.is_geocentric, ds.crs.is_geographic, ds.crs.is_projected)

False False True
False True False


In [12]:
import numpy as np
from mesoscaler.core import Mesoscale

urma = GriddedDataset.from_zarr(urma_store, [URMA.CEIL, URMA.VIS])
era5 = GriddedDataset.from_zarr(era5_store, ERA5(["temperature", "specific_humidity"]))
era5

<xarray.Dataset>
Dimensions:            (T: 2, Z: 37, Y: 721, X: 1440)
Coordinates:
    latitude           (Y, X) float32 90.0 90.0 90.0 90.0 ... -90.0 -90.0 -90.0
  * vertical           (Z) int64 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
    longitude          (Y, X) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time               (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Dimensions without coordinates: T, Z, Y, X
Data variables:
    specific_humidity  (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature        (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [13]:
from mesoscaler.core import P0

scale = Mesoscale(200, 200, pressure=[P0, 925.0, 850.0, 700.0, 500.0, 300.0], rate=20)
scale

Mesoscale:
  scale: [ 1.00  9.69 12.23 15.26 17.70 19.37]
    hpa: [1013.25  925.00  850.00  700.00  500.00  300.00]
     dx: [ 200.00 1938.97 2446.80 3051.46 3539.82 3874.94]
     dy: [ 200.00 1938.97 2446.80 3051.46 3539.82 3874.94]

In [61]:
from typing import Generator

CHANNELS = "channels"
VARIABLES = "variable"


def zip_scale(scale: Mesoscale, *dsets: GriddedDataset) -> Generator[tuple[tuple[float, ...], xr.DataArray], Any, Any]:
    all_levels = xr.concat([ds.level for ds in dsets], dim=Z)
    area_extents = scale.to_area_extents()  # [[ x0, y0, x1, y1 ], ..]

    for ds in dsets:
        levels = all_levels[all_levels.isin(ds.level)]
        extents = area_extents[np.isin(scale.hpa, levels)]

        for extents, lvl in zip(extents, levels):
            yield tuple(extents), ds.ds.sel({LVL: [lvl]}).to_array(dim=VARIABLES).transpose(Y, X, ...).stack(
                {CHANNELS: [VARIABLES, T, Z]}
            )

list(zip_scale(scale, urma, era5))


[((-200.0, -200.0, 200.0, 200.0),
  <xarray.DataArray (Y: 1597, X: 2345, channels: 4)>
  dask.array<reshape, shape=(1597, 2345, 4), dtype=float32, chunksize=(1597, 2345, 2), chunktype=numpy.ndarray>
  Coordinates:
      latitude   (Y, X) float64 19.23 19.23 19.24 19.24 ... 54.38 54.38 54.37
      longitude  (Y, X) float64 233.7 233.7 233.8 233.8 ... 300.9 300.9 301.0
    * channels   (channels) object MultiIndex
    * variable   (channels) object 'ceiling' 'ceiling' 'visibility' 'visibility'
    * time       (channels) datetime64[ns] 2019-01-02 ... 2019-01-02T01:00:00
    * vertical   (channels) float64 1.013e+03 1.013e+03 1.013e+03 1.013e+03
  Dimensions without coordinates: Y, X
  Attributes:
      area_definition:  None),
 ((-1938.972946140261,
   -1938.972946140261,
   1938.972946140261,
   1938.972946140261),
  <xarray.DataArray (Y: 721, X: 1440, channels: 4)>
  dask.array<reshape, shape=(721, 1440, 4), dtype=float32, chunksize=(721, 1440, 2), chunktype=numpy.ndarray>
  Coordinate

In [16]:
# import functools


# class ReSampler:
#     def __init__(
#         self,
#         width=80,
#         height=80,
#         *,
#         ids: IndependentDataset,
#     ) -> None:
#         md = ids.metadata
#         self._source_definition = pyresample.geometry.AreaDefinition(
#             area_id=f"{md.title} Area Definition",
#             description=md.comment,
#             proj_id=md.crs.name,
#             projection=md.crs,
#             height=ids.y.size,
#             width=ids.y.size,
#             area_extent=ids.area_extent,
#             # lons=lons,
#             # lats=lats,
#         )

#         self._target_definition = functools.partial(
#             pyresample.geometry.AreaDefinition,
#             area_id="lambert_azimuthal_equal_area",
#             description="lambert_azimuthal_equal_area",
#             proj_id="lambert_azimuthal_equal_area",
#             width=width,
#             height=height,
#         )

#     def get_target_definition(
#         self, latitude: float, longitude: float, area_extent: list[float]
#     ) -> pyresample.geometry.AreaDefinition:
#         crs = pyproj.CRS.from_cf(
#             {
#                 "grid_mapping_name": "lambert_azimuthal_equal_area",
#                 "latitude_of_projection_origin": latitude,
#                 "longitude_of_projection_origin": longitude,
#                 "units": "m",
#             }
#         )

#         return self._target_definition(projection=crs, area_extent=area_extent)


# def get_grid_definition(self: IndependentDataset) -> pyresample.geometry.GridDefinition:
#     lons = self.lons.to_numpy()
#     if is_greenwich(ds.lons.to_numpy()):
#         lons = (lons - 180) % 360 - 180

#     return pyresample.geometry.GridDefinition(lons, self.lats)


# ds = IndependentDataset.from_zarr(urma_store, URMAEnum("t2m", "d2m"))
# lons = (ds.lons.to_numpy() - 180) % 360 - 180

# from src.mesoformer.typing import Boolean


# def is_greenwich(lons: np.ndarray) -> Boolean:
#     x = lons - (lons - 180) % 180
#     return np.all(x == 180.0)


# # print(
# #     ds.lons.to_numpy(),
# #     is_greenwich(ds.lons.to_numpy()),
# #     is_greenwich(360),
# #     # ds.lons.to_numpy(),
# #     # # ds.lons.to_numpy()[0],
# #     # lons,
# #     # np.unique(lons - (lons - 180) % 360 - 180),
# #     sep="\n==\n",
# #     # ds.lats.shape,
# # )
# # # get_grid_definition(IndependentDataset.from_zarr(urma_store, URMAEnum("t2m", "d2m")))


# # print(
# #     f"""\
# # {is_greenwich(0)=}
# # {is_greenwich(180)=}
# # {is_greenwich(360)=}
# # {is_greenwich(40)=}
# # """
# # )
# def is_geodetic(crs: pyproj.CRS) -> Boolean:
#     return crs.is_geographic


# print(
#     "\n=========\n".join(
#         f"""
# {crs.name=}
# area_of_use:\n{crs.area_of_use or 'undefined'}
# {is_geodetic(crs)=}
# """
#         for crs in (ERA5Enum.crs, URMAEnum.crs)
#     )
# )

In [17]:
# from src.mesoformer.typing import StrPath, Sequence, DictStrAny, Mapping, Hashable, Iterable
# from src.mesoformer.datasets.metadata import MetadataMixin
# from src.mesoformer.datasets.metadata import T, Z, X, Y, LON, LAT, LVL, TIME, DatasetMetadata
# import functools

# VariableLike = type[CFDatasetEnum] | CFDatasetEnum | Sequence[CFDatasetEnum]


# class PartialAreaDefinitions:
#     def __init__(
#         self,
#         width=256,
#         height=256,
#         *,
#         metadata: DatasetMetadata,
#         latitude: float = 25.0,
#         longitude: float = 265.0,
#     ) -> None:
#         self._source = None

#         self._target = functools.partial(
#             pyresample.geometry.AreaDefinition,
#             area_id="lambert_azimuthal_equal_area",
#             description="lambert_azimuthal_equal_area",
#             proj_id="lambert_azimuthal_equal_area",
#             projection=pyproj.CRS.from_cf(
#                 {
#                     "grid_mapping_name": "lambert_azimuthal_equal_area",
#                     "latitude_of_projection_origin": latitude,
#                     "longitude_of_projection_origin": longitude,
#                 }
#             ),
#             width=width,
#             height=height,
#         )

#     def __repr__(self) -> str:
#         return f"{self.__class__.__name__}({self._target})"

#     def __call__(self, area_extent: list[float]):
#         tgt = self._target(area_extent=area_extent)
#         return tgt, tgt

#     def iter_definitions(
#         self, area_extents: Iterable[list[float]]
#     ) -> Iterable[tuple[pyresample.geometry.AreaDefinition, pyresample.geometry.AreaDefinition]]:
#         for area_extent in area_extents:
#             yield self(area_extent)


# area_defs = PartialAreaDefinitions(
#     80,
#     80,
#     metadata=DatasetMetadata.from_title("ERA5"),
#     latitude=25.0,
#     longitude=265.0,
# )


# for src, tgt in area_defs.iter_definitions(extents):
#     print(src, tgt)

In [18]:
# class ReSampler:
#     def __init__(
#         self,
#         area_defs: PartialAreaDefinitions,
#     ) -> None:
#         pass

In [19]:
# class MesoDataset(IndependentDataset):
#     cf = types.MappingProxyType(
#         {
#             "geographic_crs_name": "NDFD CONUS 2.5km Lambert Conformal Conic",
#             "projected_crs_name": "NDFD",
#             "semi_major_axis": 6378137.0,
#             "semi_minor_axis": 6356752.31424518,
#             "inverse_flattening": 298.25722356301,
#             "reference_ellipsoid_name": "WGS 84",
#             "longitude_of_prime_meridian": 0.0,
#             "prime_meridian_name": "Greenwich",
#             "horizontal_datum_name": "WGS84",
#             "latitude_of_projection_origin": 20.191999,
#             "longitude_of_projection_origin": 238.445999,
#             "false_easting": 0.0,
#             "false_northing": 0.0,
#         }
#     )

#     def __init__(self, ds: xr.Dataset, dvars: VariableLike) -> None:
#         super().__init__(ds, dvars)

#     #     lons, lats = self.lons.to_numpy(), self.lats.to_numpy()
#     #     area_extent = [lons.min(), lats.min(), lons.max(), lats.max()]
#     #     self._source_definition = pyresample.geometry.AreaDefinition(
#     #         self.cf["geographic_crs_name"],
#     #         "National Digital Forecast Database Grid",
#     #         self.cf["projected_crs_name"],
#     #         self.get_crs("lambert_conformal_conic", standard_parallel=25),
#     #         self.x.size,
#     #         self.y.size,
#     #         area_extent=area_extent,
#     #         lons=lons,
#     #         lats=lats,
#     #     )

#     # def get_source_definition(self) -> pyresample.geometry.AreaDefinition:
#     #     return self._source_definition

#     # def get_target_definition(
#     #     self, latitude: float, longitude: float, width: float, height: float, area_extent: list[float]
#     # ) -> pyresample.geometry.AreaDefinition:
#     #     crs = self.get_crs("lambert_azimuthal_equal_area", latitude=latitude, longitude=longitude)
#     #     return pyresample.geometry.AreaDefinition(
#     #         "target_projection",
#     #         "description",
#     #         None,
#     #         crs,
#     #         width=width,
#     #         height=height,
#     #         area_extent=area_extent,
#     #     )

#     # def get_crs(
#     #     self, grid_mapping_name: str, *, latitude: float | None = None, longitude: float | None = None, **kwargs
#     # ) -> pyproj.CRS:
#     #     origin: dict[str, Any] = {"grid_mapping_name": grid_mapping_name, "units": "m"}
#     #     if latitude is not None:
#     #         origin["latitude_of_projection_origin"] = latitude

#     #     if longitude is not None:
#     #         origin["longitude_of_projection_origin"] = longitude

#     #     return pyproj.CRS.from_cf(self.cf | origin | kwargs)

#     # # @property
#     # # def area_extent(self) -> list[float]:
#     # #     return [
#     # #         self.lons.min(),
#     # #         self.lats.min(),
#     # #         self.lons.max(),
#     # #         self.lats.max(),
#     # #     ]
#     # def _resample_on_center(self, target: pyresample.geometry.AreaDefinition) -> np.ndarray:
#     #     return pyresample.kd_tree.resample_nearest(self.area_definition, self.to_numpy(), target, radius_of_influence=50000)

#     # def resample_on_center(
#     #     self,
#     #     longitude: float,
#     #     latitude: float,
#     #     *,
#     #     width=256,
#     #     height=256,
#     #     dx=100,
#     #     dy=100,
#     #     scale_x=1,
#     #     scale_y=1,
#     #     units="km",
#     # ):
#     #     if units == "km":
#     #         dx *= 1000
#     #         dy *= 1000

#     #     dx /= 2
#     #     dy /= 2

#     #     height *= scale_y
#     #     width *= scale_x

#     #     source = self.get_source_definition()
#     #     data = self.da.to_numpy()

#     #     area_extent = [-dx * scale_x, -dy * scale_y, dx * scale_x, dy * scale_y]
#     #     target = self.get_target_definition(latitude, longitude, width, height, area_extent)

#     #     return pyresample.kd_tree.resample_nearest(source, data, target, radius_of_influence=50000)


# cf = MesoDataset.from_zarr(urma_store, URMAEnum("ceiling", "vis"))
# cf

In [20]:
# ceil = cf.dvars[0]
# print(ceil)
# ceil.crs

In [21]:
# import matplotlib.pyplot as plt


# class Dataset:
#     """
#     # CONUS and Northern Hemisphere Grids

#     https://graphical.weather.gov/docs/ndfdSRS.htm#:~:text=The%20NDFD%20uses%20the%20World%20Geodetic%20System,1984%20%28WGS84%29%20ellipsoid%20for%20its%20horizontal%20datum.

#     Grid Parameter	    CONUS 2.5km
#     Number of Points	2953665
#     Projection Type	    Lambert Conformal
#     Shape of Earth      Sphere
#     Earth Radius	    6371.2 km
#     Number of Points on the parallel	2145
#     Number of Points on the Meridian	1377
#     Latitude1:	20.191999
#     Longitude1:	238.445999
#     u/v vectors relative to:	easterly/northerly
#     Dx	2539.703 m
#     Dy	2539.703 m
#     GRIB2 grid, scan mode	64 (0100)
#     Scan i/x direction	positive
#     Scan j/y direction	positive
#     Consecutive points in	i/x direction
#     Adjacent rows scan in	same direction
#     Mesh Latitude	25
#     Orientation Longitude	265
#     Which Pole is on the Plane	north
#     Is Projection Bi-polar	no
#     Tangent Latitude1	25
#     Tangent Latitude2	25
#     Southern Latitude	-90
#     Southern Longitude	0
#     """

#     cf = types.MappingProxyType(
#         {
#             "geographic_crs_name": "NDFD CONUS 2.5km Lambert Conformal Conic",
#             "projected_crs_name": "NDFD",
#             "semi_major_axis": 6378137.0,
#             "semi_minor_axis": 6356752.31424518,
#             "inverse_flattening": 298.25722356301,
#             "reference_ellipsoid_name": "WGS 84",
#             "longitude_of_prime_meridian": 0.0,
#             "prime_meridian_name": "Greenwich",
#             "horizontal_datum_name": "WGS84",
#             "latitude_of_projection_origin": 20.191999,
#             "longitude_of_projection_origin": 238.445999,
#             "false_easting": 0.0,
#             "false_northing": 0.0,
#         }
#     )

#     def __init__(self, ds: xr.Dataset):
#         self.da = da = ds.to_array().transpose(X, Y, ...)
#         self.lons = lons = (da["longitude"].to_numpy() + 180) % 360 - 180
#         self.lats = lats = da["latitude"].to_numpy()

#         self._source_definition = pyresample.geometry.AreaDefinition(
#             self.cf["geographic_crs_name"],
#             "National Digital Forecast Database Grid",
#             self.cf["projected_crs_name"],
#             self.get_crs("lambert_conformal_conic", standard_parallel=25),
#             da[Y].size,
#             da[X].size,
#             area_extent=self.area_extent,
#             lons=lons,
#             lats=lats,
#         )

#     def get_source_definition(self) -> pyresample.geometry.AreaDefinition:
#         return self._source_definition

#     def get_target_definition(
#         self, latitude: float, longitude: float, width: float, height: float, area_extent: list[float]
#     ) -> pyresample.geometry.AreaDefinition:
#         crs = self.get_crs("lambert_azimuthal_equal_area", latitude=latitude, longitude=longitude)
#         return pyresample.geometry.AreaDefinition(
#             "target_projection",
#             "description",
#             None,
#             crs,
#             width=width,
#             height=height,
#             area_extent=area_extent,
#         )

#     def get_crs(
#         self, grid_mapping_name: str, *, latitude: float | None = None, longitude: float | None = None, **kwargs
#     ) -> pyproj.CRS:
#         origin: dict[str, Any] = {"grid_mapping_name": grid_mapping_name, "units": "m"}
#         if latitude is not None:
#             origin["latitude_of_projection_origin"] = latitude

#         if longitude is not None:
#             origin["longitude_of_projection_origin"] = longitude

#         return pyproj.CRS.from_cf(self.cf | origin | kwargs)

#     @property
#     def area_extent(self) -> list[float]:
#         return [
#             self.lons.min(),
#             self.lats.min(),
#             self.lons.max(),
#             self.lats.max(),
#         ]

#     def resample_on_center(
#         self,
#         longitude: float,
#         latitude: float,
#         *,
#         width=256,
#         height=256,
#         dx=100,
#         dy=100,
#         scale_x=1,
#         scale_y=1,
#         units="km",
#     ):
#         if units == "km":
#             dx *= 1000
#             dy *= 1000

#         dx /= 2
#         dy /= 2

#         height *= scale_y
#         width *= scale_x

#         source = self.get_source_definition()
#         data = self.da.to_numpy()

#         area_extent = [-dx * scale_x, -dy * scale_y, dx * scale_x, dy * scale_y]
#         target = self.get_target_definition(latitude, longitude, width, height, area_extent)

#         return pyresample.kd_tree.resample_nearest(source, data, target, radius_of_influence=50000)


# dataset = Dataset(get_era5().isel(T=0, Z=0))
# data = dataset.resample_on_center(longitude=-89.835, latitude=38.54)
# H, W, C = data.shape

# fig, axes = plt.subplots(1, C, figsize=(20, 5))

# for i in range(C):
#     ax = axes[i]
#     ax.imshow(data[:, :, i], origin="upper", cmap="terrain")
#     ax.set_xticks([])
#     ax.set_yticks([])